In [ ]:
%pip install pandas
%pip install numpy

In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def df_stats(df):
    print(df.info())
    # print(sessions.describe())
    print(df.head())


In [5]:
sessions_path = 'content/sessions.jsonl'

sessions = pd.read_json(file_path, lines=True, encoding='utf-8')

df_stats(sessions)

tracks_path = 'content/tracks.jsonl'

tracks = pd.read_json(tracks_path, lines=True, encoding='utf-8')

df_stats(tracks)


users_path = 'content/users.jsonl'

users = pd.read_json(users_path, lines=True, encoding='utf-8')

df_stats(users)

artists_path = 'content/artists.jsonl'

artists = pd.read_json(artists_path, lines=True, encoding='utf-8')

df_stats(artists)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453169 entries, 0 to 1453168
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype                    
---  ------      --------------    -----                    
 0   timestamp   1453169 non-null  datetime64[ns, UTC+02:00]
 1   user_id     1453169 non-null  int64                    
 2   track_id    1453169 non-null  object                   
 3   event_type  1453169 non-null  object                   
 4   session_id  1453169 non-null  int64                    
dtypes: datetime64[ns, UTC+02:00](1), int64(2), object(2)
memory usage: 55.4+ MB
None
                         timestamp  user_id                track_id  \
0        2022-01-17 12:03:00+02:00     1507  39GRCEbFDzuRoA2o3L6GCv   
1 2022-01-17 12:05:40.376000+02:00     1507  39GRCEbFDzuRoA2o3L6GCv   
2 2022-01-17 12:05:40.376000+02:00     1507  67TCzZwItIKvgZp1quRx19   
3 2022-01-17 12:08:12.883000+02:00     1507  1vx7gxUjwQnVXK2D7M44j4   
4 2022-01-17 12:11:0

In [ ]:
# A = 1
for a in 
# result = df.loc[df['Attribute2'] == 'A', 'Attribute1'].values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129648 entries, 0 to 129647
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                129648 non-null  object 
 1   artist_id         129648 non-null  object 
 2   name              129648 non-null  object 
 3   popularity        129648 non-null  int64  
 4   duration_ms       129648 non-null  int64  
 5   explicit          129648 non-null  int64  
 6   release_date      129648 non-null  object 
 7   danceability      129648 non-null  float64
 8   energy            129648 non-null  float64
 9   key               129648 non-null  int64  
 10  mode              25930 non-null   float64
 11  loudness          129648 non-null  float64
 12  speechiness       129648 non-null  float64
 13  acousticness      129648 non-null  float64
 14  instrumentalness  129648 non-null  float64
 15  liveness          129648 non-null  float64
 16  valence           12

In [ ]:
@TODO muszę połączyć dane z tracks i z sessions żeby określić ile user słucha czego
@ założe że przy skipie nie wysłuchał do końca i wywalam wtedy 1 z odpaleń

# sessions["id"] = range(0,len(sessions))
# sessions['song_id__session_id'] = sessions['track_id'].apply(lambda x: str(x)) + sessions['session_id'].apply(lambda x: str(x))

sessions['full_play'] = sessions['event_type'].apply(lambda x: 1 if x == "Like" or x == "Play" else 0)
sessions['Like'] = sessions['event_type'].apply(lambda x: 1 if x == "Like" else 0)
sessions['Play'] = sessions['event_type'].apply(lambda x: 1 if x == "Play" else 0)
sessions['Skip'] = sessions['event_type'].apply(lambda x: 1 if x == "Skip" else 0)



df_stats(sessions)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453169 entries, 0 to 1453168
Data columns (total 11 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   timestamp            1453169 non-null  datetime64[ns, UTC+02:00]
 1   user_id              1453169 non-null  int64                    
 2   track_id             1453169 non-null  object                   
 3   event_type           1453169 non-null  object                   
 4   session_id           1453169 non-null  int64                    
 5   id                   1453169 non-null  int64                    
 6   song_id__session_id  1453169 non-null  object                   
 7   full_play            1453169 non-null  int64                    
 8   Like                 1453169 non-null  int64                    
 9   Play                 1453169 non-null  int64                    
 10  Skip                 1453169 non-null  int

In [51]:
sessions = sessions.sort_values(by=['session_id', 'timestamp'])



In [52]:
result = sessions.groupby(['track_id', 'session_id']).count()

# Display the result
print(result)

                                   timestamp  user_id  event_type  id  \
track_id               session_id                                       
                       126                 6        6           6   6   
                       127                 7        7           7   7   
                       128                 3        3           3   3   
                       129                 2        2           2   2   
                       130                 4        4           4   4   
...                                      ...      ...         ...  ..   
7zzbfi8fvHe6hm342GcNYl 34683               1        1           1   1   
                       35165               1        1           1   1   
                       36169               2        2           2   2   
                       36918               3        3           3   3   
                       37981               2        2           2   2   

                                   song_id__sessio

In [53]:
print(sessions.groupby("song_id__session_id").apply())

TypeError: GroupBy.apply() missing 1 required positional argument: 'func'

In [ ]:


nr_songs_played = sessions.groupby('user_id').agg(
        {
                'id':'size',
                'event_type':[lambda x: 1 if skip]
        }
        ).reset_index()

print(nr_songs_played)

      user_id    id
0         101   216
1         102   300
2         103   310
3         104   552
4         105   221
...       ...   ...
2995     3096   499
2996     3097   502
2997     3098   413
2998     3099   500
2999     3100  1012

[3000 rows x 2 columns]


(3000, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           3000 non-null   int64 
 1   name              3000 non-null   object
 2   city              3000 non-null   object
 3   street            3000 non-null   object
 4   favourite_genres  3000 non-null   object
 5   premium_user      3000 non-null   bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 120.2+ KB
None
           user_id
count  3000.000000
mean   1600.500000
std     866.169729
min     101.000000
25%     850.750000
50%    1600.500000
75%    2350.250000
max    3100.000000
